# **Sovits (inference with f0)**

Author：Francis Hu

E-mail：francishr@whu.edu.cn

QQ：2235306122

BILIBILI：https://space.bilibili.com/636704927

Repository：https://github.com/Francis-Komizu/Sovits

# Set up

In [ ]:
!git clone https://github.com/Francis-Komizu/Sovits
%cd Sovits
!pip install -r requirements.txt
%cd monotonic_align
!python setup.py build_ext --inplace
%cd ..
!mkdir results

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
import torchaudio
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import numpy as np
import librosa

from JDC.model import JDCNet

import commons
import utils
from data_utils import UnitAudioLoader, UnitAudioCollate
from models import SynthesizerTrn
import requests

from scipy.io.wavfile import write

In [ ]:
to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300)
mean, std = -4, 4

def preprocess(wave):
    wave_tensor = torch.from_numpy(wave).float()
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor

# Load models

## Load content encoder

In [ ]:
hubert = torch.hub.load("bshall/hubert:main", "hubert_soft")

/usr/local/lib/python3.7/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/bshall/hubert/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://github.com/bshall/hubert/releases/download/v0.1/hubert-soft-0d54a1f4.pt" to /root/.cache/torch/hub/checkpoints/hubert-soft-0d54a1f4.pt


  0%|          | 0.00/361M [00:00<?, ?B/s]

## Load F0 model

In [ ]:
F0_model = JDCNet(num_class=1, seq_len=192)
params = torch.load("JDC/bst.t7", map_location='cpu')['net']
F0_model.load_state_dict(params)
_ = F0_model.eval()
F0_model = F0_model

## Load generator

In [ ]:
!gdown --id 'your model link' --output generator_accelerator.pth # you may change it

hps = utils.get_hparams_from_file("path/to/config")

net_g = SynthesizerTrn(
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint("path/to/generator", net_g, None)

## Load audio

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving natsume_0030.wav to natsume_0030.wav


In [ ]:
source, sr = torchaudio.load("path/to/wav")
source = source.unsqueeze(0)

## Convert voice

Set `f0_feat` as `None` for models trained without f0 feature!

In [ ]:
with torch.inference_mode():
    # Extract speech units
    unit = hubert.units(source)
    unit = torch.FloatTensor(unit)
    unit_lengths = torch.LongTensor([unit.size(1)])
    # Extract f0 feature
    f0_feat = F0_model.get_feature_GAN(source_mel.unsqueeze(0))
    # f0_feat = None
    # Synthesize audio
    audio = net_g.infer(unit, unit_lengths, F0=f0_feat, noise_scale=.667, noise_scale_w=0.8, length_scale=0.8)[0][0,0].data.float().numpy()

print("Source:")
ipd.display(ipd.Audio(source.squeeze(), rate=hps.data.sampling_rate))
print("Converted:")
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))

# References

https://github.com/bshall/acoustic-model

https://github.com/jaywalnut310/vits